In [9]:
#### add multiple crews with various flows
import os
os.environ["OPENAI_API_KEY"] = "NA" # This satisfies the validator
os.environ["OTEL_SDK_DISABLED"] = "true"


# install the necessary libraries 
# !pip install crewai crewai_tools langchain_community
#pip install nest_asyncio
# Warning control
import warnings
warnings.filterwarnings('ignore')

import nest_asyncio  #async functions
nest_asyncio.apply()


In [10]:
llm = "ollama/llama3.2"
llm = "ollama/mistral"


In [11]:
from crewai import Agent, Task, Crew, LLM
from crewai.flow.flow import Flow, listen, start
from crewai_tools import DirectoryReadTool, FileReadTool
import re
import os
import sys

In [ ]:



class DynamicKnowledgeFlow(Flow):
    @start()
    def list_files(self):
        print("--- Step 1: Scanning Directory ---")
        dir_tool = DirectoryReadTool(directory='./knowledge')
        files_string = dir_tool.run()
        
        # IMPROVED PARSING LOGIC:
        # DirectoryReadTool returns formatted string like:
        # "File paths: \n-./knowledge/file1.txt\n-./knowledge/file2.txt"
        # Note: File paths are prefixed with "-" (dash)
        lines = [f.strip() for f in files_string.split('\n') if f.strip()]
        
        # Filter out header lines and extract actual file paths
        file_paths = []
        for line in lines:
            # Skip header lines like "File paths:", "Files:", etc.
            if line.lower().startswith('file') and ':' in line:
                continue
            
            # Remove dash prefix if present (DirectoryReadTool uses "-./path" format)
            path = line.strip()
            if path.startswith('-'):
                path = path[1:].strip()
            
            # Remove any leading/trailing quotes
            path = path.strip('"\'')
            
            # Check if line looks like a file path
            if path.startswith('./') or path.startswith('/') or '/' in path:
                # Validate path exists or is a valid relative path
                if os.path.exists(path) or path.startswith('./'):
                    file_paths.append(path)
        
        print(f"Found {len(file_paths)} files: {file_paths}")
        return file_paths[0] if file_paths else None

    @listen(list_files)
    def read_files_dynamically(self, first_file_path):
        """Step 2: Read file content. Step 3 will summarize it with a CrewAI agent."""
        if not first_file_path:
            return "No files found in directory."
        
        if not os.path.exists(first_file_path):
            return f"File not found: {first_file_path}"
            
        print(f"--- Step 2: Reading file: {first_file_path} ---")
        
        try:
            file_tool = FileReadTool(file_path=first_file_path)
            content = file_tool.run()
            print(content[:500] + "..." if len(content) > 500 else content)
            # Plain string (path + content) for next step
            return f"FILE_PATH:{first_file_path}\n\nCONTENT:\n{content}"
        except Exception as e:
            return f"FILE_PATH:{first_file_path}\n\nERROR:{str(e)}"

    @listen(read_files_dynamically)
    def summarize_content(self, read_result):
        """Step 3: Run a CrewAI agent to summarize the file content."""
        if "ERROR:" in read_result or read_result == "No files found in directory.":
            return read_result
        
        # Parse CONTENT from previous step
        if "CONTENT:\n" in read_result:
            content = read_result.split("CONTENT:\n", 1)[1]
        else:
            content = read_result
        
        # Truncate very long content to avoid token limits (optional)
        max_chars = 12000
        if len(content) > max_chars:
            content = content[:max_chars] + "\n\n[... truncated for summary ...]"
        
        print("--- Step 3: Summarizing with CrewAI agent ---")
        
        summarizer_llm = LLM(model=llm)
        summarizer_agent = Agent(
            role="Summarizer",
            goal="Provide clear, concise summaries that capture the main points and key information.",
            backstory="You are an expert at reading long texts and distilling them into accurate, useful summaries.",
            llm=summarizer_llm,
            verbose=True,
        )
        summary_task = Task(
            description="Summarize the following text. Be concise; capture the main points, key facts, and any important conclusions or recommendations.\n\nText to summarize:\n{{content}}",
            agent=summarizer_agent,
            expected_output="A concise summary paragraph (or bullet points) with the main ideas and key information.",
        )
        crew = Crew(agents=[summarizer_agent], tasks=[summary_task])
        crew_result = crew.kickoff(inputs={"content": content})
        # Coerce to plain str so flow return value never triggers Rich recursion
        summary = crew_result.raw if hasattr(crew_result, "raw") else str(crew_result)
        summary = str(summary) if summary is not None else ""
        print("\n--- Summary ---")
        sys.stdout.write(summary + "\n")
        sys.stdout.flush()
        return summary

In [30]:
from crewai.flow.flow import router
import random as randomdata

class newflowWorks(Flow):
    value_remember: int = 1
    
    @start()
    def readfile(self):
        print("Starting point read")
    
    
    @listen(readfile)
    def listenfile(self):
        
        self.value_remember = randomdata.randint(0,2)
        print("Listen function is being called ", self.value_remember )
        return 1    
        
    @router(listenfile)
    def router_function(self):
        if(self.value_remember == 1):
            return self.function_call_router_1()
        else:
            return self.function_call_router_2()

    
    def function_call_router_1(self):
        print("Function call router 1", " Value= ",  self.value_remember)
        return 3
    
    def function_call_router_2(self):
        print("Function call router 2",  " Value= ", self.value_remember)
        return 4

In [33]:
flow_1 = newflowWorks()
result = flow_1.kickoff()

Flow started with ID: 81943c8f-6e66-45ae-a47b-f16e388a928c


Flow started with ID: 81943c8f-6e66-45ae-a47b-f16e388a928c


╭──────────────────────────────────────────────── Flow Execution ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Starting Flow Execution                                                                                        │
│  Name: newflowWorks                                                                                             │
│  ID: 81943c8f-6e66-45ae-a47b-f16e388a928c                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

Starting point read
Listen function is being called  1
Function call router 1  Value=  1


╭──────────────────────────────────────────────── Flow Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Flow Execution Completed                                                                                       │
│  Name: newflowWorks                                                                                             │
│  ID: 81943c8f-6e66-45ae-a47b-f16e388a928c                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
# Execute the Flow: list_files -> read_files_dynamically -> summarize_content (CrewAI agent)
import sys
flow = DynamicKnowledgeFlow()
result = flow.kickoff()
# # Extract plain string only (no str(result) on wrapper — avoids Rich/display RecursionError)
# summary_str = getattr(result, "output", None) or getattr(result, "result", None)
# if summary_str is None and isinstance(result, str):
#     summary_str = result
# if summary_str is None:
#     summary_str = "(summary not available)"
# sys.stdout.write("\n=== Final result (summary) ===\n")
# sys.stdout.write(str(summary_str) + "\n")
# sys.stdout.flush()

